In [10]:
!pip install pandas
!pip install numpy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [15]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_parquet('01-intro/yellow_tripdata_2023-01.parquet')

# Display first few rows
print(df.head())

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1         2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2         2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3         1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4         2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           0.97         1.0                  N           161           141   
1           1.10         1.0                  N            43           237   
2           2.51         1.0                  N            48           238   
3           1.90         1.0                  N           138             7   
4           1.43         1.0                  N           107            79   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [22]:
num_columns = len(df.columns)
print(f"Number of columns: {num_columns}")

Number of columns: 19


In [30]:

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# Compute duration in minutes
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
df_jan = df[df['tpep_pickup_datetime'].dt.month == 1]


std_duration = df_jan['duration'].std()
print(f"Duration: {std_duration:.2f} minutes")


Duration: 42.59 minutes


In [45]:
# Total records before filtering
total_records = len(df)

# Filter: keep durations between 1 and 60 minutes (inclusive)
df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Total after filtering
filtered_records = len(df_filtered)

# Calculate fraction
fraction = filtered_records / total_records
print(f"Fraction remaining: {fraction:.2%}")


Fraction remaining: 98.12%


In [46]:
from sklearn.feature_extraction import DictVectorizer

# Select only the relevant columns and cast to string to avoid label encoding
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)

# Convert to list of dictionaries (records format)
dicts = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Initialize and fit the DictVectorizer
dv = DictVectorizer()
X = dv.fit_transform(dicts)

# Check number of features (columns)
print(f"Feature matrix shape: {X.shape}")


Feature matrix shape: (3066766, 518)


In [49]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Target variable
y = df['duration'].values

# Train a linear regression model
model = LinearRegression()
model.fit(X, y)

# Predict on training data
y_pred = model.predict(X)

# Compute RMSE
from math import sqrt
rmse = sqrt(mean_squared_error(y, y_pred))

print(f"RMSE: {rmse:.2f}")


RMSE: 42.00


In [51]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60


In [52]:
df = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()


In [53]:
from sklearn.feature_extraction import DictVectorizer

# Convert to string to avoid label encoding
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)

dicts = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

dv = DictVectorizer()
X = dv.fit_transform(dicts)

y = df['duration'].values


In [54]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt

model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)
rmse = sqrt(mean_squared_error(y, y_pred))
print(f"RMSE: {rmse:.2f}")


RMSE: 7.65
